Imports

In [99]:
from PIL import Image
import numpy as np
import cv2
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [100]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

- Binary to Decimal Converter

In [101]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [102]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


## 2D Chaotic Map

- Hennon Map (Binary)

In [103]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

## Encoding


In [104]:
def encode(videoname, filename, hennonKey, poly):

    # Divide video into frames
    vidObj = cv2.VideoCapture(videoname) 
    success = True
    frames = []
    while success: 
        success, image = vidObj.read()
        frames.append(image)

    fps = vidObj.get(cv2.CAP_PROP_FPS)

    # Get the video's width and height.
    imageWidth = int(vidObj.get(3))
    imageHeight = int(vidObj.get(4))

    vidObj.release()

    frame_usage = [0 for i in range(len(frames))]
    
    # Set Limit
    frame_size = (len(frames[0]), len(frames[0][0]))
    frame_usage_limit = (frame_size[0] * frame_size[1] * 3) * 0.3

    # Get secret data
    filehandler = open(filename,"r")
    secret_data = "".join(filehandler.readlines())
    secret_data += "=!"

    # convert data to binary
    binary_secret_data = convertToBinary(secret_data)
    data_len = len(binary_secret_data)
    print(data_len)
    print(binary_secret_data)

    data_index = 0

    # Build LFSR to be used to choose frame
    frame_lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)
    frame_index = 0

    while data_index < data_len:

        # Choose Frame using LFSR
        frame_index = (frame_index + binaryToDecimal(frame_lfsr.state)) % len(frames)
        frame_lfsr.next()

        # If frame is already used, keep choosing next frame until unused frame is found.
        count = 0
        while frame_usage[frame_index] != 0:
            frame_index = (frame_index + binaryToDecimal(frame_lfsr.state)) % len(frames)
            frame_lfsr.next()
            count += 1
            if count > 100:
                print("frames exceeded")
                return
        print("frame", frame_index)

        cv2.imwrite('outputs/video_steg/frames/frame%s.jpg' % frame_index, frames[frame_index])
        
        # Create and load intermediate images.
        intermediate_imagehandler1 = Image.new(mode="RGB", size=frame_size)
        intermediate_image1 = intermediate_imagehandler1.load()
        intermediate_imagehandler2 = Image.new(mode="RGB", size=frame_size)
        intermediate_image2 = intermediate_imagehandler2.load()
            
        # Build chaotic map and derive some available pixels for the frame
        map = HennonMapBinary(frame_size, hennonKey)

        availablePixels = []

        for i in range(frame_size[0]):
            for j in range(0,frame_size[1]):
                if map[i][j] == 1:
                    availablePixels.append([i,j,0])
                    intermediate_image1[i,j] = tuple(frames[frame_index][i,j])
        
        intermediate_imagehandler1.save("outputs/video_steg/frames/frame%s_intermediate1.png" % str(frame_index),"PNG")
        
        # Build LFSR for pixel choice
        pixel_lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)
        pixel_index = 0

        while data_index < data_len:

            # Choose pixel using LFSR from Available pixels
            pixel_index = (pixel_index + binaryToDecimal(pixel_lfsr.state)) % len(availablePixels)
            pixel_lfsr.next()

            # If pixel is already used 3 times, keep choosing next pixel until unused pixel is found.
            count = 0
            flag = True
            while availablePixels[pixel_index][2] > 2:
                pixel_index = (pixel_index + binaryToDecimal(pixel_lfsr.state)) % len(availablePixels)
                pixel_lfsr.next()
                count += 1
                if count > 100:
                    flag = False
                    break
            if not flag:
                break
            
            '''
            EACH PIXEL CAN BE USED AT MAX 3 TIMES. WE KEEP COUNT OF NUMBER OF USES AS USAGESTATE.

            IF USAGESTATE IS 0: XOR 2 DATA BITS WITH 2 LSB OF RED VALUE
            IF USAGESTATE IS 1: XOR 2 DATA BITS WITH 2 LSB OF BLUE VALUE
            IF USAGESTATE IS 2: XOR 2 DATA BITS WITH 2 LSB OF GREEN VALUE
            '''

            # Extract row, column and usagestate values.
            row = availablePixels[pixel_index][0]
            col = availablePixels[pixel_index][1]
            usageState = availablePixels[pixel_index][2]

            # Select pixel in intermediate 2.
            intermediate_image2[row,col] = tuple(frames[frame_index][row,col])

            # Update with Red Value
            if usageState == 0:
                red = list(convertToBinary(frames[frame_index][row,col][0]))
                red[-2] = str(int(red[-2]) ^ int(binary_secret_data[data_index]))
                red[-1] = str(int(red[-1]) ^ int(binary_secret_data[data_index+1]))
                red = "".join(red)
                red = binaryToDecimal(red)
                frames[frame_index][row,col] = (red, frames[frame_index][row,col][1], frames[frame_index][row,col][2])

            # Update with Blue Value
            elif usageState == 1:
                blue = list(convertToBinary(frames[frame_index][row,col][1]))
                blue[-2] = str(int(blue[-2]) ^ int(binary_secret_data[data_index]))
                blue[-1] = str(int(blue[-1]) ^ int(binary_secret_data[data_index+1]))
                blue = "".join(blue)
                blue = binaryToDecimal(blue)
                frames[frame_index][row,col] = (frames[frame_index][row,col][0], blue, frames[frame_index][row,col][2])
            
            # Update with Green Value
            elif usageState == 2:
                green = list(convertToBinary(frames[frame_index][row,col][2]))
                green[-2] = str(int(green[-2]) ^ int(binary_secret_data[data_index]))
                green[-1] = str(int(green[-1]) ^ int(binary_secret_data[data_index+1]))
                green = "".join(green)
                green = binaryToDecimal(green)
                frames[frame_index][row,col] = (frames[frame_index][row,col][0], frames[frame_index][row,col][1], green)
            
            data_index += 2
            availablePixels[pixel_index][2] += 1
            frame_usage[frame_index] += 1
            if frame_usage[frame_index] >= frame_usage_limit:
                break
        
        cv2.imwrite('outputs/video_steg/frames/encoded_frame%s.jpg' % frame_index, frames[frame_index])

        # Save imtermediate 2 with total Used Bits after encoding with LFSR select
        intermediate_imagehandler2.save("outputs/video_steg/frames/frame%s_intermediate2.png" % str(frame_index),"PNG")
    
    fourcc = cv2.VideoWriter_fourcc('F', 'F', 'V', '1')
    video = cv2.VideoWriter("outputs/video_steg/encoded_video.avi", fourcc, fps, (imageWidth, imageHeight))
    for frame in frames:
        video.write(frame)
    video.release()

In [105]:
import json

def main():
    info = open("config.json","r")
    json_data = json.load(info)
    original_video = json_data["source"]["video_filename"]
    hennon_key = json_data["key"]["hennon_key"]
    lfsr_poly = json_data["key"]["lfsr_polynomial"]
    
    encode(original_video, "assets\input10k.txt", hennon_key, lfsr_poly)

In [106]:
main()

26208
...
frame 103
